<a href="https://colab.research.google.com/github/anjunjin/PFE-ING3-IA/blob/branch1/Tweeter_%26_Fake_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tweeter & Fake news
Notebook jupyter du projet tweeter et fake news par Arnaud Valette & Paul Planchon

run the first cell once to be sure all the librairies are installed

In [ ]:
!pip install tweepy pandas numpy matplotlib python-dotenv tqdm

In [84]:
import tweepy
import pickle
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from dotenv import dotenv_values
import requests
from IPython.display import JSON

### env vars
in a folder `../creds` put an .env file with
```
API_KEY=""
API_SECRET=""
API_TOKEN=""
```
which are the creds from twitter dev portal [twitter portal](https://developer.twitter.com/en/portal/projects/)

In [85]:
from google.colab import drive
drive.mount('/content/drive')

DATA_PATH = "/content/drive/My Drive/PFE/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
# env_var = dotenv_values("../.env")
# API_KEY    = env_var["API_KEY"]
# API_SECRET = env_var["API_SECRET"]
# API_TOKEN  = env_var["API_TOKEN"]

API_TOKEN = pd.read_csv(DATA_PATH+"Clés.csv", encoding='utf-8', sep=',')['Bearer Token'][0]
API_KEY = pd.read_csv(DATA_PATH+"Clés.csv", encoding='utf-8', sep=',')['API Key'][0]
API_SECRET = pd.read_csv(DATA_PATH+"Clés.csv", encoding='utf-8', sep=',')['API Key Secret'][0]

In [87]:
auth = tweepy.AppAuthHandler(API_KEY, API_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

## Monde website data
getting the JSON data from [le monde](https://www.lemonde.fr/webservice/decodex/updates) decodex updates feed

### categories
 - 1: Site parodique
 - 2: Fake news
 - 3: Faire attention
 - 4: OK mais il ne faut pas hésiter a croiser

In [88]:
data = requests.get("https://www.lemonde.fr/webservice/decodex/updates").json()

In [89]:
try:
    decodex_site = pd.read_csv("./monde_site.csv", index_col=0)
    with open('monde_urls.pickle', 'rb') as handle:
        site_index_to_url = pickle.load(handle)
except Exception:
    decodex_site = pd.DataFrame(requests.get("https://www.lemonde.fr/webservice/decodex/updates").json()["sites"]).T.drop(3, 1).rename(columns={0: "categorie", 1: "description", 2: "site"})
    decodex_url = requests.get("https://www.lemonde.fr/webservice/decodex/updates").json()["urls"] # urls of website, including sometimes twitter data

    # convert the url to index dict to an index to url dict
    site_index_to_url = {}
    for x, y in decodex_url.items():
        try:
            if site_index_to_url[y]:
                site_index_to_url[y].append(x)
        except Exception as e:
            site_index_to_url[y] = [x]

    # we save data to disk
    decodex_site.to_csv("monde_site.csv")
    with open('monde_urls.pickle', 'wb') as handle:
        pickle.dump(site_index_to_url, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#Si vous utilisez cette ligne, pensez à bien fermer la fenêtre de résultat pour des raison de sécurité
# decodex_site

In [129]:
# get alls twitter link from the urls of decodex
# fake_news_index = list(decodex_site[decodex_site["categorie"] == 2].index) #Ne prend que les sites de catégorie "Fake News"
fake_news_index = list(decodex_site.index) #Prend tous les sites, peut importe la catégorie
fake_news_twitter = {}

for i in fake_news_index:
    try:
        for url in site_index_to_url[int(i)]:
            if "twitter.com/" in url:
                if "?lang=fr" in url:
                    size = len(url)
                    fake_news_twitter[i] = url[12:size - 8].strip('@')
                else:
                    fake_news_twitter[i] = url[12:].strip('@')
    except Exception as e:
        pass

In [130]:
#Si vous utilisez cette ligne, pensez à bien fermer la fenêtre de résultat pour des raison de sécurité
# fake_news_twitter

In [92]:
    class TwitterUser():
        _screen_name = ""
        _id = ""
        _user_data = None
        followers = []
        following = []
        tweet = []
        like = []
        request_count = 0

        def __init__(self, sname = "", _json = {}):
            if (sname != ""):
                self._screen_name = sname
                self.get_info()
            if (_json != {}):
                self._json = _json

        def get_info(self):
            self._user_data = api.get_user(screen_name=self._screen_name)

        def get_all_data(self):
            self.get_tweets()
            # self.get_followers()
            # self.get_following()
            # self.get_like()

        def set_name(self, name):
            self._screen_name = name

        def set_id(self, _id):
            self._id = _id

        def get_followers(self):
            with tqdm(total=self._user_data.followers_count, desc="get_followers for " + self._user_data.screen_name) as pbar:
                for page in tweepy.Cursor(api.followers, screen_name=self._screen_name, count=200).pages():
                    self.followers.extend(page)
                    pbar.update(len(page))


        def get_following(self):
            with tqdm(total=self._user_data.friends_count, desc="get_following for " + self._user_data.screen_name) as pbar:
                for page in tweepy.Cursor(api.friends, screen_name=self._screen_name, count=200).pages():
                    self.following.extend(page)
                    pbar.update(len(page))

        # on ne peut prendre que les 3000 derniers
        def get_tweets(self):
            if self._user_data.protected:
                print("can't get tweets from " + self._user_data.screen_name + ", the account is protected")
            else:
                with tqdm(total=3000, desc="get_tweets for " + self._user_data.screen_name) as pbar:
                    for page in tweepy.Cursor(api.user_timeline, screen_name=self._screen_name, count=200).pages():
                        self.tweet.extend(page)
                        pbar.update(len(page))

        # on ne peut prendre que les 3000 derniers
        def get_like(self):
            if self._user_data.protected:
                print("can get like from " + self._user_data.screen_name + ", the account is protected")
            else:
                with tqdm(total=3000, desc="get_likes for " + self._user_data.screen_name) as pbar:
                    for page in tweepy.Cursor(api.favorites, screen_name=self._screen_name, count=200).pages():
                        self.like.extend(page)
                        pbar.update(len(page))


In [ ]:
# test = TwitterUser(sname="EuroScoop_FR")
# test.get_all_data()

In [94]:
# test.tweet[0].text

In [133]:
import pickle

with open(DATA_PATH+"PaucBas.pickle", "wb") as obj_file:
  for site in fake_news_twitter:
    user = TwitterUser(sname= fake_news_twitter[site])
    user.get_all_data()
    pickle.dump([user], obj_file, -1)
    # print(fake_news_twitter[site])

get_tweets for france_soir: 3210it [00:07, 430.60it/s]
get_tweets for ThierryRegenere:  12%|█▎        | 375/3000 [00:00<00:05, 458.22it/s]


TweepError: ignored

In [95]:
import pickle

with open(DATA_PATH+"PaucBas.pickle", "wb") as obj_file:
    pickle.dump([test], obj_file, -1)

In [96]:
import pickle

with open(DATA_PATH+"PaucBas.pickle", "rb") as obj_file:
    test = pickle.load(obj_file)

In [101]:
status_list = test[0].tweet

# test4 = json.dumps(test3[0]._json)

# type(test4)

test2 = [status._json['text'] for status in status_list]
len(test2)

3200

In [98]:
objects = []
with (open(DATA_PATH+"PaucBas.pickle", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

objects

[[<__main__.TwitterUser at 0x7f8285daed50>]]

In [99]:
import json

with open(DATA_PATH+"links.json", "w") as file: 
    file.write(json.dumps(fake_news_twitter))